In [131]:
import pandas as pd
import numpy as np
import os
import json
import csv
from bs4 import BeautifulSoup as bs
import pymongo
from splinter.exceptions import ElementDoesNotExist
from splinter import Browser
from datetime import date, timedelta
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

## Basic questions

1. Check methods used in the function
    * https://www.programiz.com/python-programming/methods/list/extend
    * https://www.programiz.com/python-programming/methods/built-in/filter
    * https://www.programiz.com/python-programming/methods/dictionary/get
2. Ran function
3. Check errors
    * The first one was that profile was not defined
    * After defining profile the second one was 'Keyerror': 'email'
2. What is profile within the function? I assume that **profile** refers to a dictionary provided in the exercise.
3. Print **emails** in different parts of the function
    
* After checking the points described above, I found that the error may be when accessing to the value of email. Following the schema provided, the way to access to email value would be **profile['document'][0]['address']['email']**(the dictionary is defined as profile). If, for example, profile had more dictionaries, these would be filtered according to the profile_id.

In [181]:
profile = {
    "profile_id": "1f5833f4e027e0a127f1161450370fa6370a8f5e",
    "registry_location": "mx",
    "full_name": "Juan Perez",
    "document": [
        {
            "type": "house|work|school",
            "address": {
                "full_address": "Benito Juarez #35, colonia Arboledas, Queretaro, Santiago de Queretaro, Mexico CP. 76000",
                "address_line_1": "Benito Juarez #35",
                "address_line_2": "colonia Arboledas",
                "city": "Nuevo Toledo",
                "foad": "Toledo",
                "soad": "",
                "postal_code": "76000",
                "country": "Mexico",
                "phone": ["4420000001","4420000000"],
                "fax": [],
                "email": ["example@domain.mx"]
            },
            "number": "",
            "description": ""
        }
      ],
    "status": "",
    "ad_hoc": []
}

In [182]:
def harmonize(profile_id):
    """Return list of emails in profile"""
    
    emails = profile['document'][0]['address']['email']
    print(emails)
    if not emails:
        emails = ''
    print(emails)
    
    for document in profile.get('document', []):
        emails.extend(document.get('address', {}).get('email', []))
    print(emails)
    
    emails = list(set(filter(None, emails)))
    print(emails)
    print('email done')
    return {'email': emails}

In [183]:
harmonize('1f5833f4e027e0a127f1161450370fa6370a8f5e')

['example@domain.mx']
['example@domain.mx']
['example@domain.mx', 'example@domain.mx']
['example@domain.mx']
email done


{'email': ['example@domain.mx']}

## Exercise 1

1. Iterate over multiple files
    * https://realpython.com/working-with-files-in-python/
    * I put some jsons as an example
2. Read each json and append it to list to have a list of dictionaries
    * https://www.geeksforgeeks.org/read-json-file-using-python/
3. Once all jsons are in a list I have to filter them
    * First filter the profiles of type 'work'
    * Using map(), I obtained the ids and addresses in combination with a lambda function specifying the required values(id, address)
    * Zip these values to further add them to the tsv file
4. Add the data to the tsv file
    * https://realpython.com/read-write-files-python/
    * https://realpython.com/python-csv/
    * As is a .tsv the delimiter is '\t'
    * Add the specified headers
    * Add the zipped values defined above


In [12]:
people_profiles = []
output_file = 'Files/work_profiles.tsv'

In [13]:
for file in os.listdir("Files"): #Path given in the exercise
    #print(file)
    with open(f'Files/{file}') as f:
        data = json.load(f)
    people_profiles.append(data)
    
filtered_profiles = list(filter(lambda x: x['document'][0]['type']=='work', people_profiles))
ids = list(map((lambda x :x['profile_id']), filtered_profiles))
addresses = list(map((lambda x :x['document'][0]['address']['full_address']), filtered_profiles))
values = zip(ids, addresses)

with open("Files/work_profiles.tsv", "w") as datafile:
    writer = csv.writer(datafile, delimiter='\t')
    writer.writerow(['profile_id', 'address'])
    writer.writerows(values) 


File4.json
File3.json
File2.json
File1.json


## Exercise 2

* Pages to obatain the data, in this case was the one provided
* How is the structure of the information?
* The form of the url involves year, month, date, hour, arrival/departure
    * Hour - Three hours(0, 6, 12, 18)(https://www.geeksforgeeks.org/python-datetime-timedelta-function/)
    * Date - Previous two days(current date - 2 & current date - 1)
    * arrival/departure
* Instructions for the browser:
    * https://splinter.readthedocs.io/en/latest/
    * https://www.crummy.com/software/BeautifulSoup/bs4/doc/
    * https://www.geeksforgeeks.org/python-lambda-anonymous-functions-filter-map-reduce/
    * Obtain the href of each record. Because in the table doesn't contain the information I want(status and time)
    * Once I have all the urls I have to iterate over them to extract the information I want.

In [257]:
#Departures

def links_flights(day, type_flight):
    """
    Return the links of either yesterday or before yesterday, and 
    departure or arrival data
    
    day - 'yesterday' / 'before_yesterday'
    type_flight - 'departure' / 'arrival'
    
    """
    
    hours = [0,6,12,18]
    today = date.today()
    yesterday = today - timedelta(1)
    before_yesterday = today - timedelta(2)
    
    
    data_days = {
        'yesterday':[yesterday.year, yesterday.month, yesterday.day],
        'before_yesterday': [before_yesterday.year,
             before_yesterday.month, before_yesterday.day]
    }
    
    links = []
    
    for hour in hours:

        base_url = f'https://www.flightstats.com/v2/flight-tracker/{type_flight}/MEX/?year={data_days[day][0]}&month={data_days[day][1]}&date={data_days[day][2]}&hour={hour}'
        
        print(base_url)
        
        executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
        browser = Browser("chrome", **executable_path, headless=False)

        browser.visit(base_url)
        html_page = browser.html
        soup_page = bs(html_page, 'html.parser')
        
        browser.find_by_xpath('//*[@id="__next"]/div/section/div/div[2]/div[1]/div[3]/div/div/div[6]')
        nav_pages = soup_page.find_all('div', class_='pagination__PageNavigationContainer-s1515b5x-1 djrWkq')

        len_pages = [x for x in nav_pages[0]]
        
        num_pages = int(len_pages[-3].text)
        
        print(num_pages)
        
        for x in range(1,(num_pages+1)):
            html = browser.html
            soup = bs(html, 'html.parser')
            pages = soup.find_all('a', class_='table__A-s1x7nv9w-2 flrJsE')
            refs = [page['href'] for page in pages]
            urls_flights = ['https://www.flightstats.com' + ref for ref in refs]
            links.extend(urls_flights)
            browser.find_by_xpath('//*[@id="__next"]/div/section/div/div[2]/div[1]/div[3]/div/div/div[5]').click()
            time.sleep(5)
        browser.quit()
    return links

In [261]:
links_1 = links_flights('yesterday', 'departures')

https://www.flightstats.com/v2/flight-tracker/departures/MEX/?year=2021&month=1&date=20&hour=0
2
https://www.flightstats.com/v2/flight-tracker/departures/MEX/?year=2021&month=1&date=20&hour=6
15
https://www.flightstats.com/v2/flight-tracker/departures/MEX/?year=2021&month=1&date=20&hour=12
12
https://www.flightstats.com/v2/flight-tracker/departures/MEX/?year=2021&month=1&date=20&hour=18
6


In [233]:
links_2 = links_flights('before_yesterday', 'arrivals')

2
14
12
6


In [259]:
links_3 = links_flights('yesterday', 'arrivals')

https://www.flightstats.com/v2/flight-tracker/arrivals/MEX/?year=2021&month=1&date=20&hour=0
2
https://www.flightstats.com/v2/flight-tracker/arrivals/MEX/?year=2021&month=1&date=20&hour=6
7
https://www.flightstats.com/v2/flight-tracker/arrivals/MEX/?year=2021&month=1&date=20&hour=12
15
https://www.flightstats.com/v2/flight-tracker/arrivals/MEX/?year=2021&month=1&date=20&hour=18
11


In [260]:
links_4 = links_flights('before_yesterday', 'arrivals')

https://www.flightstats.com/v2/flight-tracker/arrivals/MEX/?year=2021&month=1&date=19&hour=0
3
https://www.flightstats.com/v2/flight-tracker/arrivals/MEX/?year=2021&month=1&date=19&hour=6
8
https://www.flightstats.com/v2/flight-tracker/arrivals/MEX/?year=2021&month=1&date=19&hour=12
16
https://www.flightstats.com/v2/flight-tracker/arrivals/MEX/?year=2021&month=1&date=19&hour=18
9


In [275]:
links_1.extend(links_2)
links_1.extend(links_3)
links_1.extend(links_4)

In [276]:
len(links_1)

3428

In [282]:
flight_data = {
    'departure_city':[],
    'arrival_city':[],
    'status/time':[]
}

executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
    
for x in range(len(links_1)):
    
    browser.visit(links_1[x])
    
    try:
        status = browser.find_by_xpath('//*[@id="__next"]/div/section/div[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[2]/div[2]').text
        from_city = browser.find_by_xpath('//*[@id="__next"]/div/section/div[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/div/div[2]/div/div[1]/div/div[2]').text
        to_city = browser.find_by_xpath('//*[@id="__next"]/div/section/div[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/div/div[2]/div/div[3]/div/div[2]').text
        
         # Run only if variables are available
        if (status and from_city and to_city):
            # Print results
            print(f'Page {x} OK')
            
            flight_data['departure_city'].append(from_city)
            flight_data['arrival_city'].append(to_city)
            flight_data['status/time'].append(status)
        

    except:
        print(f'Data in page {x} not found or incomplete')
        
browser.quit()

Data in page 0 not found or incomplete
Page 1 OK
Page 2 OK
Page 3 OK
Page 4 OK
Page 5 OK
Page 6 OK
Page 7 OK
Page 8 OK
Page 9 OK
Page 10 OK
Data in page 11 not found or incomplete
Page 12 OK
Page 13 OK
Page 14 OK
Data in page 15 not found or incomplete
Page 16 OK
Page 17 OK
Page 18 OK
Page 19 OK
Page 20 OK
Page 21 OK
Page 22 OK
Page 23 OK
Page 24 OK
Page 25 OK
Page 26 OK
Data in page 27 not found or incomplete
Data in page 28 not found or incomplete
Page 29 OK
Page 30 OK
Page 31 OK
Page 32 OK
Data in page 33 not found or incomplete
Page 34 OK
Page 35 OK
Page 36 OK
Page 37 OK
Page 38 OK
Data in page 39 not found or incomplete
Data in page 40 not found or incomplete
Page 41 OK
Page 42 OK
Page 43 OK
Page 44 OK
Page 45 OK
Page 46 OK
Page 47 OK
Page 48 OK
Page 49 OK
Page 50 OK
Page 51 OK
Page 52 OK
Page 53 OK
Page 54 OK
Page 55 OK
Page 56 OK
Page 57 OK
Page 58 OK
Data in page 59 not found or incomplete
Page 60 OK
Page 61 OK
Page 62 OK
Page 63 OK
Page 64 OK
Page 65 OK
Page 66 OK
Page 67 OK
P

Page 620 OK
Page 621 OK
Page 622 OK
Page 623 OK
Page 624 OK
Page 625 OK
Page 626 OK
Page 627 OK
Page 628 OK
Page 629 OK
Page 630 OK
Page 631 OK
Page 632 OK
Page 633 OK
Page 634 OK
Page 635 OK
Page 636 OK
Page 637 OK
Page 638 OK
Page 639 OK
Data in page 640 not found or incomplete
Page 641 OK
Page 642 OK
Page 643 OK
Page 644 OK
Page 645 OK
Page 646 OK
Page 647 OK
Page 648 OK
Page 649 OK
Page 650 OK
Page 651 OK
Page 652 OK
Page 653 OK
Page 654 OK
Page 655 OK
Page 656 OK
Page 657 OK
Page 658 OK
Page 659 OK
Page 660 OK
Page 661 OK
Page 662 OK
Page 663 OK
Page 664 OK
Data in page 665 not found or incomplete
Page 666 OK
Page 667 OK
Page 668 OK
Page 669 OK
Page 670 OK
Page 671 OK
Page 672 OK
Page 673 OK
Page 674 OK
Page 675 OK
Page 676 OK
Page 677 OK
Page 678 OK
Page 679 OK
Page 680 OK
Page 681 OK
Page 682 OK
Page 683 OK
Page 684 OK
Page 685 OK
Page 686 OK
Page 687 OK
Page 688 OK
Page 689 OK
Data in page 690 not found or incomplete
Page 691 OK
Page 692 OK
Page 693 OK
Page 694 OK
Page 695 OK
P

Page 1193 OK
Page 1194 OK
Page 1195 OK
Page 1196 OK
Page 1197 OK
Page 1198 OK
Page 1199 OK
Page 1200 OK
Page 1201 OK
Page 1202 OK
Page 1203 OK
Page 1204 OK
Page 1205 OK
Page 1206 OK
Page 1207 OK
Page 1208 OK
Page 1209 OK
Page 1210 OK
Data in page 1211 not found or incomplete
Page 1212 OK
Page 1213 OK
Page 1214 OK
Page 1215 OK
Data in page 1216 not found or incomplete
Page 1217 OK
Page 1218 OK
Page 1219 OK
Page 1220 OK
Page 1221 OK
Page 1222 OK
Page 1223 OK
Page 1224 OK
Page 1225 OK
Page 1226 OK
Page 1227 OK
Page 1228 OK
Page 1229 OK
Page 1230 OK
Page 1231 OK
Page 1232 OK
Page 1233 OK
Page 1234 OK
Page 1235 OK
Data in page 1236 not found or incomplete
Page 1237 OK
Page 1238 OK
Page 1239 OK
Page 1240 OK
Data in page 1241 not found or incomplete
Page 1242 OK
Page 1243 OK
Page 1244 OK
Page 1245 OK
Page 1246 OK
Page 1247 OK
Page 1248 OK
Page 1249 OK
Page 1250 OK
Page 1251 OK
Page 1252 OK
Page 1253 OK
Page 1254 OK
Page 1255 OK
Page 1256 OK
Page 1257 OK
Page 1258 OK
Page 1259 OK
Page 1260 OK


Page 1763 OK
Page 1764 OK
Page 1765 OK
Data in page 1766 not found or incomplete
Page 1767 OK
Page 1768 OK
Page 1769 OK
Page 1770 OK
Page 1771 OK
Page 1772 OK
Page 1773 OK
Page 1774 OK
Page 1775 OK
Page 1776 OK
Page 1777 OK
Page 1778 OK
Page 1779 OK
Page 1780 OK
Page 1781 OK
Page 1782 OK
Page 1783 OK
Page 1784 OK
Page 1785 OK
Page 1786 OK
Page 1787 OK
Page 1788 OK
Page 1789 OK
Page 1790 OK
Data in page 1791 not found or incomplete
Page 1792 OK
Page 1793 OK
Page 1794 OK
Page 1795 OK
Page 1796 OK
Page 1797 OK
Page 1798 OK
Page 1799 OK
Page 1800 OK
Page 1801 OK
Page 1802 OK
Page 1803 OK
Page 1804 OK
Page 1805 OK
Page 1806 OK
Page 1807 OK
Page 1808 OK
Page 1809 OK
Page 1810 OK
Page 1811 OK
Page 1812 OK
Page 1813 OK
Page 1814 OK
Page 1815 OK
Data in page 1816 not found or incomplete
Page 1817 OK
Page 1818 OK
Page 1819 OK
Page 1820 OK
Page 1821 OK
Page 1822 OK
Page 1823 OK
Page 1824 OK
Page 1825 OK
Page 1826 OK
Page 1827 OK
Page 1828 OK
Page 1829 OK
Page 1830 OK
Page 1831 OK
Page 1832 OK
Pag

Page 2378 OK
Page 2379 OK
Page 2380 OK
Page 2381 OK
Page 2382 OK
Page 2383 OK
Page 2384 OK
Page 2385 OK
Page 2386 OK
Page 2387 OK
Page 2388 OK
Page 2389 OK
Page 2390 OK
Page 2391 OK
Page 2392 OK
Page 2393 OK
Page 2394 OK
Page 2395 OK
Page 2396 OK
Page 2397 OK
Page 2398 OK
Page 2399 OK
Page 2400 OK
Page 2401 OK
Page 2402 OK
Page 2403 OK
Page 2404 OK
Page 2405 OK
Page 2406 OK
Page 2407 OK
Page 2408 OK
Page 2409 OK
Page 2410 OK
Page 2411 OK
Page 2412 OK
Page 2413 OK
Page 2414 OK
Page 2415 OK
Page 2416 OK
Page 2417 OK
Page 2418 OK
Page 2419 OK
Page 2420 OK
Page 2421 OK
Page 2422 OK
Page 2423 OK
Page 2424 OK
Page 2425 OK
Page 2426 OK
Page 2427 OK
Page 2428 OK
Page 2429 OK
Page 2430 OK
Page 2431 OK
Page 2432 OK
Page 2433 OK
Page 2434 OK
Page 2435 OK
Page 2436 OK
Page 2437 OK
Page 2438 OK
Page 2439 OK
Page 2440 OK
Page 2441 OK
Page 2442 OK
Page 2443 OK
Page 2444 OK
Page 2445 OK
Page 2446 OK
Page 2447 OK
Page 2448 OK
Page 2449 OK
Page 2450 OK
Page 2451 OK
Page 2452 OK
Page 2453 OK
Page 2454 OK

Page 3004 OK
Page 3005 OK
Page 3006 OK
Page 3007 OK
Page 3008 OK
Page 3009 OK
Page 3010 OK
Page 3011 OK
Page 3012 OK
Page 3013 OK
Page 3014 OK
Page 3015 OK
Page 3016 OK
Page 3017 OK
Page 3018 OK
Page 3019 OK
Page 3020 OK
Page 3021 OK
Page 3022 OK
Page 3023 OK
Page 3024 OK
Page 3025 OK
Page 3026 OK
Page 3027 OK
Page 3028 OK
Page 3029 OK
Page 3030 OK
Page 3031 OK
Page 3032 OK
Page 3033 OK
Page 3034 OK
Page 3035 OK
Page 3036 OK
Page 3037 OK
Page 3038 OK
Page 3039 OK
Page 3040 OK
Page 3041 OK
Page 3042 OK
Page 3043 OK
Page 3044 OK
Page 3045 OK
Page 3046 OK
Page 3047 OK
Page 3048 OK
Page 3049 OK
Page 3050 OK
Page 3051 OK
Page 3052 OK
Page 3053 OK
Page 3054 OK
Page 3055 OK
Page 3056 OK
Page 3057 OK
Page 3058 OK
Page 3059 OK
Page 3060 OK
Page 3061 OK
Page 3062 OK
Page 3063 OK
Page 3064 OK
Page 3065 OK
Page 3066 OK
Page 3067 OK
Page 3068 OK
Page 3069 OK
Page 3070 OK
Page 3071 OK
Page 3072 OK
Page 3073 OK
Page 3074 OK
Page 3075 OK
Page 3076 OK
Page 3077 OK
Page 3078 OK
Page 3079 OK
Page 3080 OK

In [312]:
flight_data

{'departure_city': ['Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico City',
  'Mexico Cit

In [338]:
flights_df = pd.DataFrame(flight_data)

In [339]:
departure_delayed_flights = flights_df.loc[(flights_df['status/time']!='On time') & (flights_df['departure_city']=='Mexico City')]

In [340]:
arrivals_delayed_flights = flights_df.loc[(flights_df['status/time']!='On time') & (flights_df['arrival_city']=='Mexico City')]

In [341]:
num_departure_delayed_flights = len(departure_delayed_flights.index) #faster
num_arrival_delayed_flights = len(arrivals_delayed_flights.index)
print(f'Number of departure delayed flights: {num_departure_delayed_flights}')
print(f'Number of arrival delayed flights:{num_arrival_delayed_flights}')

Number of departure delayed flights: 25
Number of arrival delayed flights:128


In [342]:
departure_delayed_flights[['status', 'time']] = departure_delayed_flights['status/time'].str.split(" by ", expand=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [343]:
departure_delayed_flights['minutes'] = departure_delayed_flights['time'].str[-3:]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [344]:
departure_delayed_flights['hours'] = departure_delayed_flights['time'].apply(lambda s: s.split('h')[0]+'h' if 'h' in s else "0")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [345]:
departure_delayed_flights

,departure_city,arrival_city,status/time,status,time,minutes,hours
0,Mexico City,Santo Domingo,Delayed by 1h 29m,Delayed,1h 29m,29m,1h
10,Mexico City,Houston,Delayed by 4h 36m,Delayed,4h 36m,36m,4h
27,Mexico City,Houston,Delayed by 4h 36m,Delayed,4h 36m,36m,4h
386,Mexico City,Tuxtla Gutierrez,Delayed by 46m,Delayed,46m,46m,0
387,Mexico City,Tuxtla Gutierrez,Delayed by 46m,Delayed,46m,46m,0
396,Mexico City,Puerto Vallarta,Delayed by 17m,Delayed,17m,17m,0
398,Mexico City,Puerto Vallarta,Delayed by 17m,Delayed,17m,17m,0
399,Mexico City,Puerto Vallarta,Delayed by 17m,Delayed,17m,17m,0
402,Mexico City,Tampico,Delayed by 18m,Delayed,18m,18m,0
403,Mexico City,Tampico,Delayed by 18m,Delayed,18m,18m,0


### I didn't have time to finish all the points in this exercise. I just needed to clean the dataframe and obtain the data required, I almost finished. Any feedback is appreciated. Thank you